Load libraries and modules

In [ ]:
import keras
import numpy as np
from keras.applications import densenet
from keras.models import Sequential,Model,load_model
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, Convolution2D, MaxPooling2D
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from keras import regularizers, optimizers
from keras.optimizers import Adam
from keras import backend as K
import pandas as pd
import numpy as np
from glob import glob
import csv
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from PIL import Image
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sn
import pickle
from keras.models import model_from_json
from keras.models import load_model

#load densetnet121 model 
#densenet_model = densenet.DenseNet121(weights='imagenet')

In [ ]:
K.set_learning_phase(1)

In [ ]:
path = "../input/indoor-train/data_train/Data_train/"
df = pd.read_csv(path+"training.csv")
def append_ext(fn):
    return fn+".png"
df["ImageName"]=df["ImageName"].apply(append_ext)

df.head()



Process images with ImageGenerator, split into Validation and training, Augument data

In [ ]:
training_data_dir = path+"training_images"

datagen = ImageDataGenerator(rescale=1./255.,validation_split=0.25)
train_generator=datagen.flow_from_dataframe(
dataframe=df,
directory=training_data_dir,
x_col="ImageName",
y_col="ImageLabel",
subset="training",
batch_size=20,
seed=42,
zoom_range=0.1,
brightness_range=[0.2,1.0],
shuffle=True,
class_mode="categorical",
target_size=(224,224))   

valid_generator=datagen.flow_from_dataframe(
dataframe=df,
directory=training_data_dir,
x_col="ImageName",
y_col="ImageLabel",
subset="validation",
batch_size=20,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(224,224))    

In [ ]:

img_width, img_height = 224, 224
nb_train_samples = 12001
nb_validation_samples = 4000
epochs = 50
batch_size = 20
n_classes = 6

Define layers for models 

In [ ]:

def build_model():
    base_model = densenet.DenseNet121(input_shape=(224, 224, 3),
                                     weights='../input/fullkeraspretrainednotop/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                     include_top=False,
                                     pooling='avg')
    for layer in base_model.layers:
          layer.trainable = True

    x = base_model.output
    x = Dense(200, kernel_regularizer=regularizers.l1_l2(0.01), activity_regularizer=regularizers.l2(0.01))(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)
    
    x = Dense(250, kernel_regularizer=regularizers.l1_l2(0.01), activity_regularizer=regularizers.l2(0.01))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)
    
    x = Dense(200, kernel_regularizer=regularizers.l1_l2(0.01), activity_regularizer=regularizers.l2(0.01))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(n_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    
    return model

create the model and compile setting the loss and optimization function and the metrics.

In [ ]:
def euclidean_distance_loss(y_true, y_pred):
    """
    Euclidean distance loss
    https://en.wikipedia.org/wiki/Euclidean_distance
    :param y_true: TensorFlow/Theano tensor
    :param y_pred: TensorFlow/Theano tensor of the same shape as y_true
    :return: float
    """
    return K.sqrt(K.sum(K.square(y_pred - y_true), axis=-1))

opt = Adam(lr= 0.001)
model = build_model()
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['acc', 'mse'])

Early stopping if the model can't improve the loss for the validation dataset and reduce learning rate if this loss can't improve.

In [ ]:
filepath = "../input/fullkeraspretrainednotop/saved-model-{epoch:02d}-{val_acc:.2f}.hdf5"
early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, min_delta=1e-4)
#mc = ModelCheckpoint('.mdl_wtsPL.hdf5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=4, verbose=1, min_delta=1e-4)
callbacks_list = [early_stop, reduce_lr]



Train the model


In [ ]:
model_history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples //batch_size,
    epochs=epochs,
    validation_data=valid_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=callbacks_list)

plot the accuracy of the model on a graph

In [ ]:
plt.figure(0)
plt.plot(model_history.history['acc'],'r')
plt.plot(model_history.history['val_acc'],'g')
plt.xticks(np.arange(0, 50, 5.0))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of Epochs")
plt.ylabel("Accuracy")
plt.title("Training Accuracy vs Validation Accuracy")
plt.legend(['train','validation'])
 
plt.figure(1)
plt.plot(model_history.history['loss'],'r')
plt.plot(model_history.history['val_loss'],'g')
plt.xticks(np.arange(0, 50, 5.0))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of Epochs")
plt.ylabel("Loss")
plt.title("Training Loss vs Validation Loss")
plt.legend(['train','validation'])

plt.figure(2)
plt.plot(model_history.history['mean_squared_error'],'r')
plt.plot(model_history.history['val_mean_squared_error'],'g')
plt.xticks(np.arange(0, 50, 5.0))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of Epochs")
plt.ylabel("MSE")
plt.title("Training Loss vs Validation Loss")
plt.legend(['train','validation'])
 
plt.show()

In [ ]:
Save the model into a directory

In [ ]:

model.save('model.h5')